# Recommender - Categorical Cross Entropy

Recommender system with categorical cross entropy loss function with cross validation

In [28]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [29]:
from helpers import load_data

DATA_TRAIN_PATH = "data/data_train.csv"
ratings = load_data(DATA_TRAIN_PATH)

DATA_TEST_PATH = "data/sampleSubmission.csv"
samples = load_data(DATA_TEST_PATH)

number of items: 10000, number of users: 1000
number of items: 10000, number of users: 1000


## Data - preprocessing

In [30]:
from helpers import convert_train

submission,_,_=convert_train(samples)
data, n_users, n_movies = convert_train(ratings)

In [31]:
# Encode class values as integers
encoder = LabelEncoder()
encoder.fit(data ['rating'])
encoder_rate = encoder.transform(data ['rating'])

dummy_rate = np_utils.to_categorical(encoder_rate)


In [32]:
X = data[['user_id', 'movie_id']].values

In [71]:
from sklearn.model_selection import train_test_split, KFold
# split of the data: 0.9 train, 0.1 test
X_train, X_test, Y_train, Y_test = train_test_split(X,dummy_rate, test_size=0.1, random_state=42)
print(X_train)

[[8109  493]
 [8589  214]
 [5668  258]
 ...
 [1147  625]
 [5947  369]
 [1068  472]]


## Neural network model 

In [65]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(10, input_dim=2, activation='relu'))
    model.add(Dense(5, activation='sigmoid'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
n_split = 10
for train_index, test_index in KFold(n_split).split(X_train):
    x_train,x_test = X_train[train_index], X_train[test_index]
    y_train,y_test = Y_train[train_index], Y_train[test_index]
    
    model = baseline_model()
    history = model.fit(x=x_train, y=y_train,  batch_size=10000, epochs=10,verbose=1, validation_data=(x_test, y_test))

Train on 953330 samples, validate on 105926 samples
Epoch 1/10
953330/953330 [==============================] - 2s 2us/step - loss: 1.9308 - accuracy: 0.0846 - val_loss: 1.9358 - val_accuracy: 0.0847
Epoch 2/10
953330/953330 [==============================] - 1s 1us/step - loss: 1.9307 - accuracy: 0.0846 - val_loss: 1.9357 - val_accuracy: 0.0848
Epoch 3/10
953330/953330 [==============================] - 1s 1us/step - loss: 1.9306 - accuracy: 0.0847 - val_loss: 1.9275 - val_accuracy: 0.0844
Epoch 4/10
953330/953330 [==============================] - 1s 1us/step - loss: 1.6694 - accuracy: 0.0467 - val_loss: 1.6688 - val_accuracy: 0.0373
Epoch 5/10
953330/953330 [==============================] - 1s 1us/step - loss: 1.6102 - accuracy: 0.0370 - val_loss: 1.6094 - val_accuracy: 0.0373
Epoch 6/10
953330/953330 [==============================] - 1s 1us/step - loss: 1.6094 - accuracy: 0.0370 - val_loss: 1.6094 - val_accuracy: 0.0373
Epoch 7/10
953330/953330 [==============================] - 

In [ ]:
from sklearn.metrics import mean_squared_error

prediction_test = np.array([a[0] for a in model.predict(x=X_test)])

print (mean_squared_error(Y_test, prediction_test))